<a href="https://colab.research.google.com/github/testgithubprecious/Ml_projects/blob/main/Deepdream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install if not already: pip install torch torchvision pillow matplotlib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# ------------------------------
# Load pre-trained model
# ------------------------------
model = models.inception_v3(pretrained=True, aux_logits=False)
model.eval()

# ------------------------------
# Select a layer to enhance
# ------------------------------
selected_layer = model.Mixed_5b  # Try Mixed_6a or others

# Hook to capture activations
activations = None
def hook_fn(module, input, output):
    global activations
    activations = output

selected_layer.register_forward_hook(hook_fn)

# ------------------------------
# Image preprocessing / deprocessing
# ------------------------------
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

def deprocess(img_tensor):
    img = img_tensor.detach().cpu().clone()
    img = img.squeeze().permute(1, 2, 0).numpy()
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    return img

# ------------------------------
# Load image
# ------------------------------
image_path = "sample.jpg"  # Replace with your image path
image = Image.open(image_path).convert('RGB')
input_tensor = preprocess(image).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
input_tensor.requires_grad_(True)

# ------------------------------
# DeepDream gradient ascent
# ------------------------------
optimizer = torch.optim.Adam([input_tensor], lr=0.01)

for i in range(30):  # Iterations
    optimizer.zero_grad()
    model(input_tensor)
    loss = activations.norm()  # Maximize activations
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"🌀 Iteration {i}, Loss: {loss.item():.4f}")

# ------------------------------
# Display DeepDream result
# ------------------------------
dream_image = deprocess(input_tensor)
plt.imshow(dream_image)
plt.axis("off")
plt.title("💭 DeepDream Image")
plt.show()